# Notebook 4: Projecting Injections

In this notebook we will explore how we can use GravyFlow GPU functions to project injections onto the GPU with physically in a physically realistic fashion. As usual we will start by performing the necessary imports:

In [1]:
# Built-in imports
from typing import List
from pathlib import Path

# Import the GravyFlow module.
import gravyflow as gf

# Dependency imports: 
import numpy as np
import jax
from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot

In order to demonstrate projection onto a detector network, we will need example waveforms. We can use the CBCGenerator we learned about in notebook 3 to generate example waveforms:

In [2]:
# Create a waveform generator to generate example IMRPhenomD waveforms:
phenom_d_generator : gf.WaveformGenerator = gf.CBCGenerator(
    mass_1_msun=50.0,
    mass_2_msun=50.0,
    inclination_radians=0.0
)
phenom_d_injection_generator : gf.InjectionGenerator = gf.InjectionGenerator(phenom_d_generator)


# Generate one exammple waveform with the generator:
phenom_d_injection, _, _ = next(phenom_d_injection_generator(num_examples_per_batch=1))

## Projecting Injections onto Multiple Detectors

GravyFlow uses the `gf.Network` class to handle networks of virtual gravitational wave detectors.

`gf.Network` has the following initialization parameters:

- `parameters` : `Union[List[IFO], Dict]`, Required
  > There are two ways to define the detector network when initializing a `gf.Network` object. A list of interferometers already hard coded in to GravyFlow, or a dictionary containing custom interferometers.

- `seed` : `int` = `None`
  > The seed for the random number generators used when projecting injections. If set to None, the seed from `gf.Defaults` will be used.

First, we will initlise a detector object with a list of detectors already hard coded into GravyFlow:

In [3]:
# Generate a gf.Network object that constists of the Livinston (L1), Hanford (H1), and Virgo (V1) detectors.
network : gf.Network = gf.Network(
    parameters=[gf.IFO.L1, gf.IFO.H1, gf.IFO.V1]
)

With this `gf.Network` object, we can project the injections that we previously generated, using the `project_wave` class function, which has the following arguments:

- `strain` : `jax.Array`, Required
  > A `jax.Array` containing batches of both polarizations of raw strain of the injection that you wish to project.

- `sample_rate_hertz` : `Optional[float]` = `None`, Optional
  > The sample rate in Hertz of the input strain. Defaults to the value sample rate set in `gf.Defaults`.

- `right_ascension` : `Optional[Union[jax.Array, List[float], float]]` = `None`, Optional
  > The right ascesnion of the simulated source of the gravitational wave, which will be used for the projection of the injection. By default, the sources will be randomly distributed across the sky.

- `declination` : `Optional[Union[jax.Array, List[float], float]]` = `None`, Optional
  > The declination of the simualted source of the gravitational wave, which will be used for the projection of the injection. By default, the sources will be randomly distributed across the sky.

-  `polarization` : `Optional[Union[jax.Array, List[float], float]]` = `None`, Optional
  > The polarizaton of the simulated source of the gravitational wave, which will be used for the projection of the injection. By default, the sources will be randomly distributed across the sky.

Let's project the world using a toy value of 0.0 for all of the three sky location and polarization parameters: `right_ascension`, `declination`, `polarization`.

In [4]:
# Project the waveform contained in phenom_d_injection onto the detectors 
# defined in network.projection_wave:
projected_injections : jax.Array = network.project_wave(
    phenom_d_injection[0],
    right_ascension=0.0,
    declination=0.0,
    polarization=0.0
)

We can then plot the output of the waveform projection, the projection will create one channel for each detector, and will no longer contain both polarizations as they have been reduced in the projection.

In [5]:
# Since we are only projecting one injection, extract the first injection:
projected_injection : jax.Array = projected_injections[0]

# Plot the three channel output of the projection.
projection_layout : List = [
    [gf.generate_strain_plot(
        {"Injection Test": injection},
        title=f"Injection projection example"    
    )]
    for injection in projected_injection
]

# Arrange the plots in a grid layout and display them in the notebook.
grid = gridplot(projection_layout)
output_notebook()
show(grid)

Loading BokehJS ...

## Projecting from a random Sky Location and Polarization:

Rather than inputing a specific sky localisation and polarization and direction. The default behaviour of `project_wave` is to project the waveform from a random sky direction and polarization.

We can see that below:

In [6]:
# Random direction and polarisation
projected_injections : jax.Array = network.project_wave(
    phenom_d_injection[0]
)

projected_injection : jax.Array = projected_injections[0]
projection_layout : List = [
[gf.generate_strain_plot(
    {"Injection Test": injection},
    title=f"Injection projection example"    
)]
for injection in projected_injection
]

# Arrange the plots in a grid layout and display them in the notebook.
grid = gridplot(projection_layout)
output_notebook()
show(grid)

Loading BokehJS ...

## Adding custom detectors

We can also add custom detectors using a dictionary, which we can see in the example below:

In [7]:
# Adding Custom Detectors:

# Create from dictionary:
network = gf.Network({
    "longitude_radians" : [np.pi/2, np.pi/4], 
    "latitude_radians" : [-np.pi/4, np.pi/6],
    "y_angle_radians" : [(2*np.pi/3), np.pi], 
    "x_angle_radians" : None, 
    "height_meters" : [0.0, 0.0],
    "x_length_meters" : [4000.0, 10000.0],
    "y_length_meters" : [4000.0, 10000.0]
})

# Random direction and polarisation
projected_injections : jax.Array = network.project_wave(
    phenom_d_injection[0]
)

projected_injection : jax.Array = projected_injections[0]

projection_layout = [
[gf.generate_strain_plot(
    {"Injection Test": injection},
    title=f"Injection projection example"    
)]
for injection in projected_injection
]

# Arrange the plots in a grid layout and display them in the notebook.
grid = gridplot(projection_layout)
output_notebook()
show(grid)

Loading BokehJS ...

## Load Detector From Config

We can also load a custom network from a `.json` file, as in the example below:

In [8]:
# Define injection directory path:
example_network_directory : Path =  Path("./example_configs/example_network.json")

network = gf.Network.load(example_network_directory)

# Random direction and polarisation
projected_injections : jax.Array = network.project_wave(
    phenom_d_injection[0]
)

projected_injection : jax.Array = projected_injections[0]

projection_layout = [
[gf.generate_strain_plot(
    {"Injection Test": injection},
    title=f"Injection projection example"    
)]
for injection in projected_injection
]

# Arrange the plots in a grid layout and display them in the notebook.
grid = gridplot(projection_layout)
output_notebook()
show(grid)

Loading BokehJS ...

## Incoherent Injections

We can generate incoherent injections by using a `gf.IncoherentGenerator` composed of other waveform generators, which allows us to use a different waveform in each detector. Note that if we do this we must be carefull to use the same number of component waveforms as there are detectors in the network we use to project with.

In [9]:
wnb_generator_a : gf.WaveformGenerator = gf.WNBGenerator(
    duration_seconds=0.7,
    min_frequency_hertz=50.0,
    max_frequency_hertz=100.0
)
phenom_d_generator_a : gf.WaveformGenerator = gf.CBCGenerator(
    mass_1_msun=50.0,
    mass_2_msun=50.0,
    inclination_radians=10.0
)
phenom_d_generator_b : gf.WaveformGenerator = gf.CBCGenerator(
    mass_1_msun=10.0,
    mass_2_msun=10.0,
    inclination_radians=20.0
)

incoherent_generator : gf.InjectionGenerator  = gf.IncoherentGenerator(
    [wnb_generator_a, phenom_d_generator_a, phenom_d_generator_b]
)

incoherent_injection_generator : gf.InjectionGenerator = gf.InjectionGenerator(incoherent_generator)    

/home/michael.norman/gravyflow/gravyflow/src/dataset/features/injection.py:732: UserWarning: CBCGenerator: Inclination should be in [0, pi] radians. inclination range: [10.0, 10.0]
  warn(
/home/michael.norman/gravyflow/gravyflow/src/dataset/features/injection.py:732: UserWarning: CBCGenerator: Inclination should be in [0, pi] radians. inclination range: [20.0, 20.0]
  warn(


Finally, we can plot these incoherent injections:

In [10]:
network = gf.Network([gf.IFO.L1, gf.IFO.H1, gf.IFO.H1])

incoherent_injections, _, _ = next(
    incoherent_injection_generator(num_examples_per_batch=1)
)        

projected_injections : jax.Array = network.project_wave(
    incoherent_injections
)

projected_injection : jax.Array = projected_injections[0]

projection_layout = [
[gf.generate_strain_plot(
    {"Injection Test": injection},
    title=f"Injection projection example"    
)]
for injection in projected_injection
]

# Arrange the plots in a grid layout and display them in the notebook.
grid = gridplot(projection_layout)
output_notebook()
show(grid)

Loading BokehJS ...